**Modification:**
- Use Bidirectional LSTM 

```l1 = Bidirectional(LSTM(100, consume_less='gpu', init='glorot_uniform', return_sequences=True, dropout_W=0.2))(main_input)```

- Change from target_chars to targetchartoindice (same len but target_chars not exist) 

```act_output = Dense(len(targetchartoindice), activation='softmax', init='glorot_uniform', name='act_output')(b2_1)```

In [1]:
import pandas as pd
import numpy as np
import pickle
from utils import * 

In [2]:
from keras.models import Sequential, Model
from keras.layers.core import Dense
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.layers import Input
from keras.layers.wrappers import Bidirectional
from keras.utils.data_utils import get_file
from keras.regularizers import WeightRegularizer
from keras.optimizers import Nadam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from theano.ifelse import ifelse

Using Theano backend.


In [3]:
with open('../input/parameters.pkl', "rb") as f:
    maxlen = pickle.load(f)
    num_features = pickle.load(f)
    chartoindice = pickle.load(f)
    targetchartoindice = pickle.load(f)
    divisor = pickle.load(f)
    divisor2 = pickle.load(f)

In [4]:
with open('../input/preprocessed_data.pkl', "rb") as f:
    X = pickle.load(f)
    y_a = pickle.load(f)
    y_t = pickle.load(f)
    X_test = pickle.load(f)
    y_a_test = pickle.load(f)
    y_t_test = pickle.load(f)

In [5]:
# build the model: 
print('Build model...')
main_input = Input(shape=(maxlen, num_features), name='main_input')
# shared layer
l1 = Bidirectional(LSTM(100, consume_less='gpu', init='glorot_uniform', return_sequences=True, dropout_W=0.2))(main_input) # the shared layer
b1 = BatchNormalization()(l1)

# layers
l2_1 = Bidirectional(LSTM(75, consume_less='gpu', init='glorot_uniform', return_sequences=False, dropout_W=0.2))(b1) # the layer specialized in activity prediction
b2_1 = BatchNormalization()(l2_1)
l2_2 = Bidirectional(LSTM(75, consume_less='gpu', init='glorot_uniform', return_sequences=False, dropout_W=0.5))(b1) # the layer specialized in time prediction
b2_2 = BatchNormalization()(l2_2)

act_output = Dense(len(targetchartoindice), activation='softmax', init='glorot_uniform', name='act_output')(b2_1)
time_output = Dense(1, init='glorot_uniform', name='time_output')(b2_2)

model = Model(input=[main_input], output=[act_output, time_output])

opt = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004, clipvalue=3)

model.compile(loss={'act_output':'categorical_crossentropy', 'time_output':'mae'}, optimizer=opt)
early_stopping = EarlyStopping(monitor='val_loss', patience=83)
model_checkpoint = ModelCheckpoint('output_files/models/model_{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)

model.fit(X, {'act_output':y_a, 'time_output':y_t}, validation_split=0.2, verbose=2, callbacks=[early_stopping, model_checkpoint, lr_reducer], batch_size=maxlen, nb_epoch=500)

Build model...
Train on 7344 samples, validate on 1837 samples
Epoch 1/500
35s - loss: 2.0967 - act_output_loss: 0.8821 - time_output_loss: 1.2146 - val_loss: 1.6042 - val_act_output_loss: 0.6022 - val_time_output_loss: 1.0020
Epoch 2/500
35s - loss: 1.7034 - act_output_loss: 0.6591 - time_output_loss: 1.0443 - val_loss: 1.6677 - val_act_output_loss: 0.5923 - val_time_output_loss: 1.0753
Epoch 3/500
35s - loss: 1.6831 - act_output_loss: 0.6484 - time_output_loss: 1.0347 - val_loss: 1.6215 - val_act_output_loss: 0.6262 - val_time_output_loss: 0.9953
Epoch 4/500
34s - loss: 1.6376 - act_output_loss: 0.6262 - time_output_loss: 1.0114 - val_loss: 1.5759 - val_act_output_loss: 0.5908 - val_time_output_loss: 0.9851
Epoch 5/500
36s - loss: 1.6196 - act_output_loss: 0.6167 - time_output_loss: 1.0029 - val_loss: 1.5847 - val_act_output_loss: 0.6003 - val_time_output_loss: 0.9844
Epoch 6/500
35s - loss: 1.6312 - act_output_loss: 0.6260 - time_output_loss: 1.0052 - val_loss: 1.5448 - val_act_outp

36s - loss: 1.5414 - act_output_loss: 0.5619 - time_output_loss: 0.9795 - val_loss: 1.5120 - val_act_output_loss: 0.5671 - val_time_output_loss: 0.9449
Epoch 52/500
35s - loss: 1.5396 - act_output_loss: 0.5609 - time_output_loss: 0.9787 - val_loss: 1.5112 - val_act_output_loss: 0.5635 - val_time_output_loss: 0.9478
Epoch 53/500
35s - loss: 1.5460 - act_output_loss: 0.5669 - time_output_loss: 0.9791 - val_loss: 1.5143 - val_act_output_loss: 0.5681 - val_time_output_loss: 0.9461
Epoch 54/500
35s - loss: 1.5367 - act_output_loss: 0.5594 - time_output_loss: 0.9772 - val_loss: 1.5150 - val_act_output_loss: 0.5684 - val_time_output_loss: 0.9466
Epoch 55/500
35s - loss: 1.5396 - act_output_loss: 0.5616 - time_output_loss: 0.9780 - val_loss: 1.5135 - val_act_output_loss: 0.5675 - val_time_output_loss: 0.9461
Epoch 56/500
37s - loss: 1.5430 - act_output_loss: 0.5596 - time_output_loss: 0.9834 - val_loss: 1.5138 - val_act_output_loss: 0.5668 - val_time_output_loss: 0.9470
Epoch 57/500
35s - loss

36s - loss: 1.5352 - act_output_loss: 0.5558 - time_output_loss: 0.9794 - val_loss: 1.5122 - val_act_output_loss: 0.5671 - val_time_output_loss: 0.9452
Epoch 102/500
35s - loss: 1.5323 - act_output_loss: 0.5555 - time_output_loss: 0.9768 - val_loss: 1.5135 - val_act_output_loss: 0.5676 - val_time_output_loss: 0.9458
Epoch 103/500
34s - loss: 1.5311 - act_output_loss: 0.5520 - time_output_loss: 0.9791 - val_loss: 1.5129 - val_act_output_loss: 0.5668 - val_time_output_loss: 0.9460
Epoch 104/500
38s - loss: 1.5370 - act_output_loss: 0.5577 - time_output_loss: 0.9794 - val_loss: 1.5129 - val_act_output_loss: 0.5667 - val_time_output_loss: 0.9462
Epoch 105/500


KeyboardInterrupt: 

In [6]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
main_input (InputLayer)          (None, 15, 14)        0                                            
____________________________________________________________________________________________________
bidirectional_1 (Bidirectional)  (None, 15, 200)       92000       main_input[0][0]                 
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 15, 200)       800         bidirectional_1[0][0]            
____________________________________________________________________________________________________
bidirectional_2 (Bidirectional)  (None, 150)           165600      batchnormalization_1[0][0]       
___________________________________________________________________________________________